![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Try It Yourself: Context Engineering Experiments

## Learning Objectives (45 minutes)
By the end of this hands-on session, you will be able to:
1. **Modify** student profiles and observe how context changes affect recommendations
2. **Experiment** with different memory types and storage patterns
3. **Test** context retrieval with various queries and filters
4. **Design** context engineering solutions for your own use cases
5. **Evaluate** the impact of context quality on AI agent performance

## Prerequisites
- Completed notebooks 01, 02, and 03 in Section 1
- Environment setup verified and working
- Basic understanding of context engineering concepts

---

## Introduction

Now that you understand the fundamentals of context engineering, it's time to get hands-on! This notebook provides a playground for experimenting with the concepts we've covered:

- **Student Profile Modifications**: See how changing interests, preferences, and history affects recommendations
- **Memory Experiments**: Store different types of information and test retrieval
- **Context Retrieval Testing**: Try various queries and observe what memories are retrieved
- **Your Own Use Cases**: Apply context engineering principles to your domain

## Setup and Verification

Let's start by setting up our environment and verifying everything is working:

In [ ]:
# Environment setup
import os
import asyncio
from dotenv import load_dotenv
from datetime import datetime
import json

# Load environment variables
load_dotenv()

# Set up environment with consistent defaults
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"Redis URL: {REDIS_URL}")
print(f"Agent Memory URL: {AGENT_MEMORY_URL}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

In [ ]:
# Import required modules
try:
    from redis_context_course.models import (
        Course, StudentProfile, DifficultyLevel, CourseFormat
    )
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    
    print("✅ Core modules imported successfully")
    
    # Test Redis connection
    if redis_config.health_check():
        print("✅ Redis connection healthy")
    else:
        print("❌ Redis connection failed")
        
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure you've completed the setup notebook.")

In [ ]:
# Initialize course manager
course_manager = CourseManager()

# Quick test to ensure course data is available
try:
    test_results = await course_manager.search_courses("programming", limit=1)
    if test_results:
        print(f"✅ Course data available: Found {len(test_results)} course(s)")
        print(f"   Sample: {test_results[0].course_code} - {test_results[0].title}")
    else:
        print("⚠️  No course data found. You may need to run the data generation scripts.")
        print("   See notebook 03_setup_environment.ipynb for instructions.")
except Exception as e:
    print(f"❌ Course search failed: {e}")

## Experiment 1: Student Profile Modifications

Let's create different student profiles and see how they affect course recommendations:

### Base Student Profile

First, let's create a baseline student profile:

In [ ]:
# Create a baseline student profile
baseline_student = StudentProfile(
    name="Alex Johnson",
    email="alex.johnson@university.edu",
    major="Computer Science",
    year=2,
    completed_courses=["CS101", "MATH101", "ENG101"],
    current_courses=["CS201", "MATH201"],
    interests=["programming", "web development"],
    preferred_format=CourseFormat.HYBRID,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=15
)

print("👤 Baseline Student Profile:")
print(f"Name: {baseline_student.name}")
print(f"Major: {baseline_student.major} (Year {baseline_student.year})")
print(f"Completed: {baseline_student.completed_courses}")
print(f"Interests: {baseline_student.interests}")
print(f"Preferences: {baseline_student.preferred_format.value}, {baseline_student.preferred_difficulty.value}")

# Get baseline recommendations
try:
    baseline_recommendations = await course_manager.get_recommendations(baseline_student, limit=3)
    print(f"\n📚 Baseline Recommendations ({len(baseline_recommendations)} courses):")
    for i, course in enumerate(baseline_recommendations, 1):
        print(f"   {i}. {course.course_code}: {course.title}")
        print(f"      Format: {course.format.value}, Difficulty: {course.difficulty.value}")
except Exception as e:
    print(f"❌ Recommendation failed: {e}")

### 🧪 Your Turn: Modify the Student Profile

Now it's your turn to experiment! Try modifying different aspects of the student profile and observe how recommendations change:

In [ ]:
# Experiment 1A: Change interests
# TODO: Modify the interests list and see how recommendations change

experiment_1a_student = StudentProfile(
    name="Alex Johnson - Experiment 1A",
    email="alex.johnson@university.edu",
    major="Computer Science",
    year=2,
    completed_courses=["CS101", "MATH101", "ENG101"],
    current_courses=["CS201", "MATH201"],
    interests=["machine learning", "artificial intelligence", "data science"],  # Changed from web development
    preferred_format=CourseFormat.HYBRID,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=15
)

print("🧪 Experiment 1A: Changed Interests")
print(f"New interests: {experiment_1a_student.interests}")

try:
    exp_1a_recommendations = await course_manager.get_recommendations(experiment_1a_student, limit=3)
    print(f"\n📚 New Recommendations ({len(exp_1a_recommendations)} courses):")
    for i, course in enumerate(exp_1a_recommendations, 1):
        print(f"   {i}. {course.course_code}: {course.title}")
        print(f"      Format: {course.format.value}, Difficulty: {course.difficulty.value}")
    
    print("\n🔍 Analysis:")
    print("   Compare these recommendations with the baseline.")
    print("   How did changing interests affect the course suggestions?")
    
except Exception as e:
    print(f"❌ Recommendation failed: {e}")

# YOUR TURN: Try different interests below
# Suggestions: "cybersecurity", "game development", "mobile apps", "blockchain", "robotics"

In [ ]:
# Experiment 1B: Change format preference
# TODO: Try different course formats and see the impact

experiment_1b_student = StudentProfile(
    name="Alex Johnson - Experiment 1B",
    email="alex.johnson@university.edu",
    major="Computer Science",
    year=2,
    completed_courses=["CS101", "MATH101", "ENG101"],
    current_courses=["CS201", "MATH201"],
    interests=["programming", "web development"],
    preferred_format=CourseFormat.ONLINE,  # Changed from HYBRID
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=15
)

print("🧪 Experiment 1B: Changed Format Preference")
print(f"New format preference: {experiment_1b_student.preferred_format.value}")

try:
    exp_1b_recommendations = await course_manager.get_recommendations(experiment_1b_student, limit=3)
    print(f"\n📚 New Recommendations ({len(exp_1b_recommendations)} courses):")
    for i, course in enumerate(exp_1b_recommendations, 1):
        print(f"   {i}. {course.course_code}: {course.title}")
        print(f"      Format: {course.format.value}, Difficulty: {course.difficulty.value}")
    
    print("\n🔍 Analysis:")
    print("   Notice how format preference affects which courses are recommended.")
    print("   Are more online courses being suggested now?")
    
except Exception as e:
    print(f"❌ Recommendation failed: {e}")

# YOUR TURN: Try CourseFormat.IN_PERSON below

In [ ]:
# Experiment 1C: Your custom student profile
# TODO: Create your own student profile with different characteristics

# Template for your experiment:
your_custom_student = StudentProfile(
    name="Your Name Here",
    email="your.email@university.edu",
    major="Your Major",  # Try: "Data Science", "Information Systems", "Mathematics"
    year=1,  # Try different years: 1, 2, 3, 4
    completed_courses=[],  # Add courses you've "completed"
    current_courses=[],  # Add courses you're "taking"
    interests=["your", "interests", "here"],  # Add your actual interests
    preferred_format=CourseFormat.ONLINE,  # Choose your preference
    preferred_difficulty=DifficultyLevel.BEGINNER,  # Choose your level
    max_credits_per_semester=12  # Adjust as needed
)

print("🧪 Your Custom Student Profile:")
print(f"Name: {your_custom_student.name}")
print(f"Major: {your_custom_student.major} (Year {your_custom_student.year})")
print(f"Interests: {your_custom_student.interests}")
print(f"Preferences: {your_custom_student.preferred_format.value}, {your_custom_student.preferred_difficulty.value}")

try:
    your_recommendations = await course_manager.get_recommendations(your_custom_student, limit=5)
    print(f"\n📚 Your Personalized Recommendations ({len(your_recommendations)} courses):")
    for i, course in enumerate(your_recommendations, 1):
        print(f"   {i}. {course.course_code}: {course.title}")
        print(f"      Format: {course.format.value}, Difficulty: {course.difficulty.value}")
        print(f"      Description: {course.description[:100]}...")
    
    print("\n🤔 Reflection Questions:")
    print("   1. Do these recommendations make sense for your profile?")
    print("   2. How do they differ from the baseline recommendations?")
    print("   3. What would you change to get better recommendations?")
    
except Exception as e:
    print(f"❌ Recommendation failed: {e}")

## Experiment 2: Memory Storage and Retrieval

Now let's experiment with storing and retrieving different types of memories. This will help you understand how context accumulates over time.

### Memory Client Setup

First, let's set up the memory client (if Agent Memory Server is available):

In [ ]:
# Try to set up memory client
memory_available = False
memory_client = None

try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import MemoryTypeEnum, ClientMemoryRecord
    
    # Initialize memory client with a unique namespace for experiments
    config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL,
        default_namespace=f"experiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    memory_client = MemoryAPIClient(config=config)
    
    print(f"✅ Memory client initialized")
    print(f"   Namespace: {config.default_namespace}")
    memory_available = True
    
except ImportError:
    print("⚠️  Agent Memory Client not available")
    print("   Memory experiments will use simulated data")
except Exception as e:
    print(f"⚠️  Memory server connection failed: {e}")
    print("   Memory experiments will use simulated data")

print(f"\nMemory experiments: {'🧠 Live' if memory_available else '🎭 Simulated'}")

### 🧪 Your Turn: Store Different Memory Types

Let's experiment with storing different types of memories:

In [ ]:
# Experiment 2A: Store different types of memories
async def store_sample_memories():
    """Store various types of memories for experimentation."""
    
    if not memory_available:
        print("🎭 Simulating memory storage (Agent Memory Server not available)")
        sample_memories = [
            "Student prefers online courses due to work schedule",
            "Student struggled with calculus but excelled in programming",
            "Student wants to specialize in machine learning",
            "Student mentioned interest in startup culture",
            "Student completed CS101 with grade A"
        ]
        for i, memory in enumerate(sample_memories, 1):
            print(f"   {i}. [SIMULATED] {memory}")
        return sample_memories
    
    # Real memory storage
    memories_to_store = [
        ClientMemoryRecord(
            text="Student prefers online courses because they work part-time at a tech startup",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["preferences", "schedule", "work"]
        ),
        ClientMemoryRecord(
            text="Student struggled with calculus concepts but excelled in programming assignments",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["academic_performance", "strengths", "challenges"]
        ),
        ClientMemoryRecord(
            text="Student expressed strong interest in machine learning and AI career path",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["career_goals", "interests", "machine_learning"]
        ),
        ClientMemoryRecord(
            text="Student mentioned wanting to start their own tech company someday",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["entrepreneurship", "goals", "ambitions"]
        ),
        ClientMemoryRecord(
            text="Student completed CS101 Introduction to Programming with grade A",
            memory_type=MemoryTypeEnum.EPISODIC,
            topics=["academic_history", "achievements", "programming"]
        )
    ]
    
    try:
        result = await memory_client.create_long_term_memory(memories_to_store)
        print(f"✅ Stored {len(memories_to_store)} memories successfully")
        
        for i, memory in enumerate(memories_to_store, 1):
            print(f"   {i}. [{memory.memory_type.value}] {memory.text}")
            print(f"      Topics: {', '.join(memory.topics)}")
        
        return [m.text for m in memories_to_store]
        
    except Exception as e:
        print(f"❌ Memory storage failed: {e}")
        return []

print("🧪 Experiment 2A: Storing Sample Memories")
print("=" * 50)
stored_memories = await store_sample_memories()

In [ ]:
# Experiment 2B: Test memory retrieval with different queries
async def test_memory_retrieval(query, limit=3):
    """Test memory retrieval with a specific query."""
    
    if not memory_available:
        print(f"🎭 Simulating search for: '{query}'")
        # Simple keyword matching simulation
        relevant_memories = []
        for memory in stored_memories:
            if any(word.lower() in memory.lower() for word in query.split()):
                relevant_memories.append(memory)
        
        print(f"   Found {len(relevant_memories[:limit])} relevant memories:")
        for i, memory in enumerate(relevant_memories[:limit], 1):
            print(f"   {i}. {memory}")
        return relevant_memories[:limit]
    
    # Real memory search
    try:
        results = await memory_client.search_long_term_memory(
            text=query,
            limit=limit
        )
        
        print(f"🔍 Search results for '{query}':")
        print(f"   Found {len(results.memories)} relevant memories:")
        
        for i, memory in enumerate(results.memories, 1):
            print(f"   {i}. [{memory.memory_type}] {memory.text}")
            print(f"      Relevance: {memory.score:.3f}")
        
        return [m.text for m in results.memories]
        
    except Exception as e:
        print(f"❌ Memory search failed: {e}")
        return []

print("\n🧪 Experiment 2B: Testing Memory Retrieval")
print("=" * 50)

# Test different queries
test_queries = [
    "online courses",
    "programming skills",
    "career goals",
    "academic performance"
]

for query in test_queries:
    print(f"\n📝 Query: '{query}'")
    await test_memory_retrieval(query, limit=2)
    print("-" * 30)

In [ ]:
# Experiment 2C: Your custom memory experiments
# TODO: Try storing your own memories and testing retrieval

print("🧪 Experiment 2C: Your Custom Memory Experiments")
print("=" * 50)

# Template for your custom memories
your_custom_memories = [
    "Add your own memory here - what would you want an AI agent to remember about you?",
    "Another memory - perhaps about your learning style or preferences",
    "A third memory - maybe about your goals or interests"
]

print("💡 Ideas for custom memories:")
print("   • Learning preferences (visual, hands-on, theoretical)")
print("   • Time constraints (busy schedule, flexible hours)")
print("   • Technical background (beginner, intermediate, expert)")
print("   • Career aspirations (specific roles, industries)")
print("   • Past experiences (successes, challenges, interests)")

print("\n🔧 Your turn: Modify the 'your_custom_memories' list above and run this cell again!")

# Store your custom memories (simulated)
if your_custom_memories[0] != "Add your own memory here - what would you want an AI agent to remember about you?":
    print("\n📝 Your Custom Memories:")
    for i, memory in enumerate(your_custom_memories, 1):
        print(f"   {i}. {memory}")
    
    # Test retrieval with your custom query
    your_query = "learning"  # Change this to test different queries
    print(f"\n🔍 Testing retrieval with your query: '{your_query}'")
    
    # Simple simulation of retrieval
    relevant = [m for m in your_custom_memories if your_query.lower() in m.lower()]
    if relevant:
        print(f"   Found {len(relevant)} relevant memories:")
        for i, memory in enumerate(relevant, 1):
            print(f"   {i}. {memory}")
    else:
        print("   No memories found matching your query.")
        print("   Try a different query or add more specific memories.")
else:
    print("\n⏳ Waiting for you to add your custom memories...")
    print("   Edit the 'your_custom_memories' list above and re-run this cell.")

## Experiment 3: Context Retrieval Testing

Let's experiment with how different queries retrieve different types of context:

In [ ]:
# Experiment 3A: Course search with different query types
print("🧪 Experiment 3A: Course Search Query Testing")
print("=" * 50)

# Test different types of queries
search_queries = [
    "machine learning",           # Specific topic
    "beginner programming",       # Difficulty + topic
    "online data science",        # Format + topic
    "advanced mathematics",       # Difficulty + subject
    "web development projects",   # Topic + approach
]

for query in search_queries:
    print(f"\n📝 Query: '{query}'")
    try:
        results = await course_manager.search_courses(query, limit=2)
        print(f"   Found {len(results)} courses:")
        for i, course in enumerate(results, 1):
            print(f"   {i}. {course.course_code}: {course.title}")
            print(f"      Difficulty: {course.difficulty.value}, Format: {course.format.value}")
    except Exception as e:
        print(f"   ❌ Search failed: {e}")
    print("-" * 30)

In [ ]:
# Experiment 3B: Your custom search queries
# TODO: Try your own search queries and analyze the results

print("🧪 Experiment 3B: Your Custom Search Queries")
print("=" * 50)

# Add your own search queries here
your_queries = [
    "your search query here",
    "another query to try",
    "third query for testing"
]

print("💡 Query ideas to try:")
print("   • Your actual interests (e.g., 'cybersecurity', 'game design')")
print("   • Skill combinations (e.g., 'python data analysis', 'javascript frontend')")
print("   • Career-focused (e.g., 'software engineering', 'product management')")
print("   • Technology-specific (e.g., 'react development', 'cloud computing')")

print("\n🔧 Your turn: Modify the 'your_queries' list above with your interests!")

# Test your custom queries
if your_queries[0] != "your search query here":
    for query in your_queries:
        print(f"\n📝 Your Query: '{query}'")
        try:
            results = await course_manager.search_courses(query, limit=3)
            if results:
                print(f"   Found {len(results)} courses:")
                for i, course in enumerate(results, 1):
                    print(f"   {i}. {course.course_code}: {course.title}")
                    print(f"      {course.description[:80]}...")
            else:
                print("   No courses found. Try a broader or different query.")
        except Exception as e:
            print(f"   ❌ Search failed: {e}")
        print("-" * 40)
else:
    print("\n⏳ Waiting for your custom queries...")
    print("   Edit the 'your_queries' list above and re-run this cell.")

## Experiment 4: Design Your Own Use Case

Now it's time to think about how context engineering could apply to your own domain or use case:

### 🧪 Your Turn: Context Engineering Use Case Design

Think about a domain you're familiar with and design a context-aware AI agent for it:

In [ ]:
# Experiment 4: Design your own context engineering use case
print("🧪 Experiment 4: Your Context Engineering Use Case")
print("=" * 60)

print("💡 Use Case Ideas:")
print("   🏥 Healthcare: Patient care assistant that remembers medical history")
print("   🛒 E-commerce: Shopping assistant that learns preferences over time")
print("   📚 Learning: Personalized tutor that adapts to learning style")
print("   💼 Business: Project management assistant that tracks team context")
print("   🎵 Entertainment: Music recommendation agent with mood awareness")
print("   🏠 Smart Home: Home automation that learns daily routines")
print("   💰 Finance: Investment advisor that remembers risk tolerance")
print("   🍳 Cooking: Recipe assistant that knows dietary restrictions")

print("\n📝 Design Template:")
print("   Fill out the template below for your chosen domain:")

# Template for use case design
your_use_case = {
    "domain": "Your Domain Here (e.g., Healthcare, E-commerce, etc.)",
    "agent_purpose": "What does your agent help users accomplish?",
    "user_context": [
        "What should the agent know about users?",
        "What preferences matter?",
        "What history is important?"
    ],
    "system_context": [
        "What should the agent know about itself?",
        "What are its capabilities?",
        "What are its limitations?"
    ],
    "memory_types": [
        "What should be remembered short-term?",
        "What should be remembered long-term?",
        "What should be forgotten?"
    ],
    "tools_needed": [
        "What external data sources?",
        "What actions can it perform?",
        "What integrations are needed?"
    ]
}

print("\n🔧 Your turn: Modify the 'your_use_case' dictionary above!")
print("   Then re-run this cell to see your design.")

# Display the use case design
if your_use_case["domain"] != "Your Domain Here (e.g., Healthcare, E-commerce, etc.)":
    print("\n🎯 Your Context Engineering Use Case:")
    print("=" * 50)
    print(f"📋 Domain: {your_use_case['domain']}")
    print(f"🎯 Purpose: {your_use_case['agent_purpose']}")
    
    print("\n👤 User Context:")
    for item in your_use_case['user_context']:
        print(f"   • {item}")
    
    print("\n🤖 System Context:")
    for item in your_use_case['system_context']:
        print(f"   • {item}")
    
    print("\n🧠 Memory Strategy:")
    for item in your_use_case['memory_types']:
        print(f"   • {item}")
    
    print("\n🛠️ Tools & Integrations:")
    for item in your_use_case['tools_needed']:
        print(f"   • {item}")
    
    print("\n🤔 Reflection Questions:")
    print("   1. How would context engineering improve user experience in your domain?")
    print("   2. What are the biggest challenges for implementing this?")
    print("   3. How would you measure success?")
    print("   4. What privacy considerations are important?")
    
else:
    print("\n⏳ Waiting for your use case design...")
    print("   Edit the 'your_use_case' dictionary above and re-run this cell.")

## Reflection and Analysis

Let's reflect on what you've learned through these experiments:

### 📊 Experiment Summary

Take a moment to analyze your experimental results:

In [ ]:
# Reflection exercise
print("📊 Experiment Reflection and Analysis")
print("=" * 50)

print("🤔 Reflection Questions:")
print("\n1. Student Profile Experiments:")
print("   • How did changing interests affect course recommendations?")
print("   • Which profile changes had the biggest impact?")
print("   • What surprised you about the recommendation differences?")

print("\n2. Memory Experiments:")
print("   • How did different memory types serve different purposes?")
print("   • Which queries retrieved the most relevant memories?")
print("   • What would happen if memories were inaccurate or outdated?")

print("\n3. Context Retrieval:")
print("   • How did query phrasing affect search results?")
print("   • Which search strategies worked best for your interests?")
print("   • What would improve the relevance of results?")

print("\n4. Use Case Design:")
print("   • What context engineering challenges are unique to your domain?")
print("   • How would you handle privacy and data sensitivity?")
print("   • What would be the most valuable context to capture?")

print("\n💡 Key Insights:")
print("   • Context quality directly impacts AI agent usefulness")
print("   • Different context types serve different purposes")
print("   • Personalization requires balancing relevance and privacy")
print("   • Context engineering is domain-specific but follows common patterns")

print("\n📝 Your Insights:")
print("   Write your key takeaways in the cell below...")

### ✍️ Your Key Takeaways

**Write your insights from the experiments here:**

1. **Most Surprising Discovery:**
   *(What surprised you most about how context affects AI behavior?)*

2. **Biggest Challenge:**
   *(What was the most difficult aspect of context engineering to understand?)*

3. **Best Application Idea:**
   *(What's the most exciting use case you can imagine for context engineering?)*

4. **Next Steps:**
   *(What would you like to learn more about in the upcoming sections?)*

## Summary and Next Steps

Congratulations! You've completed hands-on experiments with context engineering fundamentals:

### ✅ What You've Accomplished

Through these experiments, you've:

1. **Explored Student Profile Impact**
   - Modified interests, preferences, and academic history
   - Observed how context changes affect recommendations
   - Understood the importance of accurate user modeling

2. **Experimented with Memory Systems**
   - Stored different types of memories (semantic, episodic)
   - Tested memory retrieval with various queries
   - Learned how context accumulates over time

3. **Tested Context Retrieval**
   - Tried different search query strategies
   - Analyzed how query phrasing affects results
   - Discovered the importance of semantic understanding

4. **Designed Your Own Use Case**
   - Applied context engineering principles to your domain
   - Identified key context types and memory strategies
   - Considered real-world implementation challenges

### 🔑 Key Principles Learned

- **Context Quality Matters**: Better context leads to better AI responses
- **Personalization is Powerful**: Individual preferences dramatically affect recommendations
- **Memory Types Serve Different Purposes**: Semantic vs. episodic memory have distinct roles
- **Retrieval Strategy is Critical**: How you search affects what context you find
- **Domain Adaptation is Essential**: Context engineering must be tailored to specific use cases

### 🚀 Ready for Section 2

You're now prepared to dive deeper into context engineering with **Section 2: System Context**, where you'll learn:

- **System Instructions**: How to craft effective prompts that define agent behavior
- **Tool Definition**: How to design and implement agent tools
- **Tool Selection Strategies**: Advanced patterns for choosing the right tools

### 💭 Keep Experimenting!

The power of context engineering lies in its ability to make AI systems more intelligent, personalized, and useful. As you continue through the course, keep experimenting with:

- Different context combinations
- Novel memory storage patterns
- Creative retrieval strategies
- Domain-specific applications

**Happy context engineering!** 🎉